Présentation de la base de données


Nous avons exporté les données de l'OCDE dans une base de données que nous avons reformatée, afin d'avoir une indexation multiple au niveau des colonnes : un premier niveau pour les pays et ensuite pour chaque pays les différents indicateurs choisis. Nous avons aussi ajouté des indicateurs concernant les inégalités, à savoir le top 50% du revenu et le top 10% du revenu. Néanmoins nous nous posons quelques questions concernant ces indicateurrs. 1) Le coefficient de Gini est disponible mais le calcul n'est fait que par année (car il nous semble qu'il varie peu). Faut-il alors qu'on le calcule à la main pour chaque trimestre, l'utiliser tel quel (par annnée), ou ne pas l'utiliser du tout? 2) Est-il important de savoir si la part des revenus est bruts ou non, dans le mesure où l'on cherche à évaluer l'impact des politiques fiscales de Trump?

In [1]:
import csv

In [9]:
import pandas as pd

ocde_df = pd.read_csv(r"https://raw.githubusercontent.com/Adrien-Mcode/Stat-app-Trumpnomics/main/Tableaux_csv/ocde2.csv?token=ASA6PWXVFPZLLJVOXZ3ZBD3AC2VAQ")

ocde_df.rename(columns={'Unnamed: 0':'Variables'}, inplace=True)

ind_tuple = list(zip(ocde_df['Pays'], ocde_df['Variables']))
new_index = pd.MultiIndex.from_tuples(ind_tuple, names=["Pays", "Variables"])

tocde = ocde_df.T.copy()
tocde.columns = new_index
tocde.drop(['Variables', 'Pays'], inplace=True)

In [5]:
import csv
bottom_50 = pd.read_csv(r"https://raw.githubusercontent.com/Adrien-Mcode/Stat-app-Trumpnomics/main/Donnees%20inegalite/top_bottom_income/WID_Data_Metadata/bottom_50_income.csv?token=ASA6PWTDLEXNQ6N2CXJQMWTAC2VEG",
                       header = 1,
                       sep=';')

In [6]:
import csv
top_10 =pd.read_csv(r"https://raw.githubusercontent.com/Adrien-Mcode/Stat-app-Trumpnomics/main/Donnees%20inegalite/top_bottom_income/WID_Data_Metadata/top_10_income.csv?token=ASA6PWQC3X44Z3RYIRG6MWDAC2VF4",
                   header = 1,
                   sep=';')

In [11]:
df_ocde = pd.read_csv(r"https://raw.githubusercontent.com/Adrien-Mcode/Stat-app-Trumpnomics/main/Tableaux_csv/ocde2.csv?token=ASA6PWXVFPZLLJVOXZ3ZBD3AC2VAQ")

In [12]:
import pandas as pd
import csv


pays_ocde = {"Germany" :'DEU',"Australia" :'AUS',"Austria":'AUT',"Belgium":'BEL',
             "Canada":'CAN',"Denmark":'DNK',"Spain":'ESP',"Finland":'FIN',
             "France":'FRA',"Greece":'GRC',"Ireland":'IRL',"Italy":'ITA',
             "Japan":'JPN',"Luxembourg":'LUX',"Norway":'NOR',"New-Zealand":'NZL',
             "Netherlands":'NLD',"Portugal":'PRT',"United Kingdom":'GBR',
             "Sweden":'SWE',"Switzerland":'CHE',"Turkey":'TUR',"United-States":'USA'}



df_ocde.rename(columns={'Unnamed: 0':'Variables'}, inplace=True)

ind_tuple = list(zip(df_ocde['Pays'], df_ocde['Variables']))
new_index = pd.MultiIndex.from_tuples(ind_tuple, names=["Pays", "Variables"])
#mise en place du système de double indice
tocde = df_ocde.T.copy()
tocde.columns = new_index
tocde.drop(['Variables', 'Pays'], inplace=True)
#tocde = tocde.drop(['2020-Q1','2020-Q2','2020-Q3','2020-Q4','2021-Q1'])
#renommage des colonnes de bottom_50 et top_10
bottom_50.columns = ['Variables', 'Year'] + sorted(pays_ocde.keys(), key=str)
top_10.columns = ['Variables', 'Year'] + sorted(pays_ocde.keys(), key=str)

#on prépare bottom_50 pour la fusion
bottom_50 = bottom_50.drop('Variables', axis=1)
for i in range (1,3) : 
    bottom_50 = pd.concat([bottom_50,bottom_50]) 
    
annee = []
for j in range (1,5):
    for i in range (0,29):
        annee.append(str(1991+i)+'-Q'+str(j))
bottom_50['Year'] = annee

#Mise en place double index pour bottom_50 :
bottom_50 = bottom_50.set_index('Year')

bottom_50['Variables'] = 'income p0p50'

ind_tuple = list(zip(bottom_50.columns, bottom_50['Variables']))

new_index = pd.MultiIndex.from_tuples(ind_tuple, names=["Pays", "Variables"])

bottom_50.columns = new_index

bottom_50 = bottom_50.drop(('Variables','income p0p50'), axis = 1)

tocde = tocde.merge(right=bottom_50, how='outer',
                    left_index=True,
                    right_index=True).sort_index(axis = 0).sort_index(axis = 1)

#on prépare top_10 pour la fusion :
top_10 = top_10.drop('Variables', axis = 1)
for i in range (1,3) : 
    top_10 = pd.concat([top_10,top_10])    
    
annee = []
for j in range (1,5):
    for i in range (0,29):
        annee.append(str(1991+i)+'-Q'+str(j))
top_10['Year'] = annee

#Mise en place double index pour top_10 :
top_10 = top_10.set_index('Year')

top_10['Variables'] = 'income p90p100'

ind_tuple = list(zip(top_10.columns, top_10['Variables']))

new_index = pd.MultiIndex.from_tuples(ind_tuple, names=["Pays", "Variables"])

top_10.columns = new_index

top_10 = top_10.drop(('Variables','income p90p100'), axis=1)

tocde = tocde.merge(right = top_10,
                    how='outer',
                    left_index=True,
                    right_index=True).sort_index(axis = 0).sort_index(axis = 1)

qs = tocde.index.str.replace(r'(Q\d) (\d+)', r'\2-\1')

tocde['date'] = pd.PeriodIndex(qs, freq='Q').to_timestamp()


tocde = tocde.reindex(index=tocde['date'])
tocde = tocde.drop("date", axis=1)

ValueError: Length mismatch: Expected axis has 23 elements, new values have 25 elements

In [8]:
tocde

Pays            AUS                                                         \
Variables    Actifs  Chomage    Conso  Emplois  Exports Formation      PIB   
date                                                                         
1990-01-01  63.5595  6.23002  108.373  59.5996   26.942    41.272   206.89   
1990-04-01  63.7865  6.46453  108.965  59.6628   26.986    39.657  207.173   
1990-07-01  63.9287  7.22537  109.106  59.3098   28.052    39.505  205.931   
1990-10-01  63.6977  7.86751  108.421  58.6863   28.496    38.098  207.139   
1991-01-01  63.3995  8.73256  108.655  57.8632   29.718    36.525  204.453   
...             ...      ...      ...      ...      ...       ...      ...   
2018-10-01   65.629  5.03693  269.315  62.3233  116.359   113.548  486.394   
2019-01-01  65.6701   5.0256   270.33  62.3697  117.741   112.894  488.812   
2019-04-01  65.9566  5.22984  271.244  62.5071  119.861   111.682  491.985   
2019-07-01  66.0785   5.2299  271.195  62.6227  120.854   112.746  494.974   
2019-10-01  65.9389  5.16322  272.211  62.5343  120.146   112.018  496.921   

Pays            AUT                    ...      USA                    \
Variables    Actifs  Chomage    Conso  ...  Chomage    Conso  Emplois   
date                                   ...                              
1990-01-01      NaN  3.15392      NaN  ...      5.3  5998.06     63.2   
1990-04-01      NaN  3.26997      NaN  ...  5.33333  6016.33       63   
1990-07-01      NaN  3.41228      NaN  ...      5.7  6040.16  62.6667   
1990-10-01      NaN  3.23917      NaN  ...  6.13333  5994.21  62.3333   
1991-01-01      NaN  3.38382      NaN  ...      6.6  5971.67     61.9   
...             ...      ...      ...  ...      ...      ...      ...   
2018-10-01  61.4248  4.77307  47.4263  ...  3.83333  13033.4     60.5   
2019-01-01  61.5643  4.76479  48.1421  ...  3.86667  13093.2     60.7   
2019-04-01  61.4554  4.47055  47.8964  ...  3.66667  13212.8  60.6333   
2019-07-01  61.2008  4.37813  47.7142  ...      3.6  13301.3  60.8333   
2019-10-01  61.2359  4.33111  47.5393  ...      3.6  13353.7  60.9667   

Pays                                   United Kingdom                 \
Variables   Exports Formation      PIB   income p0p50 income p90p100   
date                                                                   
1990-01-01    682.3   1695.37  9358.29            NaN            NaN   
1990-04-01  690.785   1668.44  9392.25            NaN            NaN   
1990-07-01  696.014   1659.14   9398.5            NaN            NaN   
1990-10-01  702.143   1628.39  9312.94            NaN            NaN   
1991-01-01  706.434   1587.41  9269.37         0.2823         0.2593   
...             ...       ...      ...            ...            ...   
2018-10-01  2548.79   3983.12  18813.9         0.2559         0.2894   
2019-01-01  2560.41   4020.35  18950.3         0.2564         0.2884   
2019-04-01  2531.42   4029.71  19020.6         0.2564         0.2884   
2019-07-01  2536.56   4054.51  19141.7         0.2564         0.2884   
2019-10-01  2557.85    4075.4    19254         0.2564         0.2884   

Pays       United-States                 
Variables   income p0p50 income p90p100  
date                                     
1990-01-01           NaN            NaN  
1990-04-01           NaN            NaN  
1990-07-01           NaN            NaN  
1990-10-01           NaN            NaN  
1991-01-01        0.2496         0.2931  
...                  ...            ...  
2018-10-01        0.2368         0.3187  
2019-01-01        0.2367         0.3187  
2019-04-01        0.2367         0.3187  
2019-07-01        0.2367         0.3187  
2019-10-01        0.2367         0.3187  

[120 rows x 207 columns]

*QUESTION* : Que faut-il faire des valeurs manquantes indiquée par NaN? Les remplacer par 0 fausserait les résultalts

Nous allons désormais effectuer des tests de stationnarité sur les différentes variables. Nous appliquons deux tests  : KPSS qui a pour hypothèse nulle H0:"La série est stationnaire" et le test AFD dont l'hypothèse nulle est H0 : "La série est non stationnaire". NB : nous n'avons pas désaisonnalisé les séries avant d'effectuer les tests car nous ne savions pas si cela éait nécessaire.

 Dans un premier temps nous nous intéressons au PIB des Etats-Unis. D'abord le test KPSS :

In [122]:
serie1=tocde['USA']['PIB']

date
1990-01-01    9358.29
1990-04-01    9392.25
1990-07-01     9398.5
1990-10-01    9312.94
1991-01-01    9269.37
               ...   
2018-10-01    18813.9
2019-01-01    18950.3
2019-04-01    19020.6
2019-07-01    19141.7
2019-10-01      19254
Name: PIB, Length: 120, dtype: object

In [123]:
from statsmodels.tsa.stattools import kpss
def kpss_test(series, **kw):    
    statistic, p_value, n_lags, critical_values = kpss(series, **kw)
    # Format Output
    print(f'KPSS Statistic: {statistic}')
    print(f'p-value: {p_value}')
    print(f'num lags: {n_lags}')
    print('Critial Values:')
    for key, value in critical_values.items():
        print(f'   {key} : {value}')
    print(f'Result: The series is {"not " if p_value < 0.05 else ""}stationary')

kpss_test(serie1)

KPSS Statistic: 0.9556147530640225
p-value: 0.01
num lags: 13
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
Result: The series is not stationary


C:\Users\duran\anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1661: FutureWarning: The behavior of using lags=None will change in the next release. Currently lags=None is the same as lags='legacy', and so a sample-size lag length is used. After the next release, the default will change to be the same as lags='auto' which uses an automatic lag length selection method. To silence this warning, either use 'auto' or 'legacy'
  warn(msg, FutureWarning)
C:\Users\duran\anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1685: InterpolationWarning: p-value is smaller than the indicated p-value
  warn("p-value is smaller than the indicated p-value", InterpolationWarning)


La série n'est pas stationnaire au seuil de 5%, nous allons donc appliquer le test à la série intégrée à l'ordre 1.

In [124]:
serie2 = serie1 - serie1.shift().fillna(0)
serie2.drop(pd.Timestamp('1990-01-01 00:00:00'), inplace=True)
kpss_test(serie2)

KPSS Statistic: 0.08667738001103942
p-value: 0.1
num lags: 13
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
Result: The series is stationary


C:\Users\duran\anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1687: InterpolationWarning: p-value is greater than the indicated p-value
  warn("p-value is greater than the indicated p-value", InterpolationWarning)


La série intégrée à l'ordre 1 est stationnaire selon le test KPSS. Maintenant, effectuons le test ADF.

In [125]:
from statsmodels.tsa.stattools import adfuller
def adf_test(series, **kw):    
    statistic, p_value, n_lags, nobs, critical_values, icbest = adfuller(series, **kw)
    # Format Output
    print(f'ADF Statistic: {statistic}')
    print(f'p-value: {p_value}')
    print(f'num lags: {n_lags}')
    print('Critial Values:')
    for key, value in critical_values.items():
        print(f'   {key} : {value}')
    print(f'Result: The series is {"not " if p_value > 0.05 else ""}stationary')

adf_test(serie1)
adf_test(serie2)

ADF Statistic: -0.03066086875171472
p-value: 0.9559218024233863
num lags: 2
Critial Values:
   1% : -3.487517288664615
   5% : -2.8865777180380032
   10% : -2.5801239192052012
Result: The series is not stationary
ADF Statistic: -5.015879029087158
p-value: 2.0642028929741757e-05
num lags: 1
Critial Values:
   1% : -3.487517288664615
   5% : -2.8865777180380032
   10% : -2.5801239192052012
Result: The series is stationary


Comme pour le test KPSS, la série intégrée à l'ordre 1 est stationnaire.
Maintenant effectuons les tests sur le chômage

In [20]:
serie1=tocde['USA']['Chomage']

In [25]:
kpss_test(serie1)

KPSS Statistic: 0.09403467015765274
p-value: 0.1
num lags: 13
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
Result: The series is stationary


C:\Users\duran\anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1687: InterpolationWarning: p-value is greater than the indicated p-value
  warn("p-value is greater than the indicated p-value", InterpolationWarning)


In [24]:
adf_test(serie1)

ADF Statistic: -2.257662581670269
p-value: 0.18598624704216277
num lags: 9
Critial Values:
   1% : -3.4912451337340342
   5% : -2.8881954545454547
   10% : -2.5809876033057852
Result: The series is not stationary


In [27]:
serie2 = serie1 - serie1.shift().fillna(0)

In [28]:
adf_test(serie2)

ADF Statistic: -3.2664589070402466
p-value: 0.016445973000680066
num lags: 7
Critial Values:
   1% : -3.4901313156261384
   5% : -2.8877122815688776
   10% : -2.5807296460459184
Result: The series is stationary


La série du chomage des Etats-Unis est stationnaire d'après le test KPSS, nous n'avons pas besoin de l'intégrer à l'ordre 1. Cependant elle n'est pas stationnaire selon le test AFD. Elle est stationnaire selon le test AFD intégrée à l'ordre 1. Maintenant intéressons nous à l'emploi.

In [30]:
serie1=tocde['USA']['Emplois']

In [31]:
kpss_test(serie1)

KPSS Statistic: 0.562377817476475
p-value: 0.027617608676469594
num lags: 13
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
Result: The series is not stationary


In [32]:
serie2 = serie1 - serie1.shift().fillna(0)
kpss_test(serie2)

KPSS Statistic: 0.34232633403073226
p-value: 0.1
num lags: 13
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
Result: The series is stationary


C:\Users\duran\anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1687: InterpolationWarning: p-value is greater than the indicated p-value
  warn("p-value is greater than the indicated p-value", InterpolationWarning)


In [33]:
adf_test(serie1)

ADF Statistic: -1.767806019245896
p-value: 0.39655813857383915
num lags: 3
Critial Values:
   1% : -3.4880216384691867
   5% : -2.8867966864160075
   10% : -2.5802408234244947
Result: The series is not stationary


In [34]:
adf_test(serie2)

ADF Statistic: -4.378986843724712
p-value: 0.00032330793316602985
num lags: 2
Critial Values:
   1% : -3.487517288664615
   5% : -2.8865777180380032
   10% : -2.5801239192052012
Result: The series is stationary


La série temporelle de l'emploi aux Etats-Unis intégrée à l'ordre 1 est stationnaire selon les test KPSS et AFD.

In [35]:
serie1=tocde['USA']['Actifs']

In [37]:
kpss_test(serie1)
adf_test(serie1)
serie2 = serie1 - serie1.shift().fillna(0)
kpss_test(serie2)
adf_test(serie2)

KPSS Statistic: 0.7821858654012013
p-value: 0.01
num lags: 13
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
Result: The series is not stationary
ADF Statistic: -0.5665598786841386
p-value: 0.8783605257395495
num lags: 6
Critial Values:
   1% : -3.489589552580676
   5% : -2.887477210140433
   10% : -2.580604145195395
Result: The series is not stationary
KPSS Statistic: 0.3443191824462999
p-value: 0.1
num lags: 13
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
Result: The series is stationary
ADF Statistic: -3.3176396689354313
p-value: 0.014108159521618074
num lags: 5
Critial Values:
   1% : -3.489057523907491
   5% : -2.887246327182993
   10% : -2.5804808802708528
Result: The series is stationary


C:\Users\duran\anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1685: InterpolationWarning: p-value is smaller than the indicated p-value
  warn("p-value is smaller than the indicated p-value", InterpolationWarning)
C:\Users\duran\anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1687: InterpolationWarning: p-value is greater than the indicated p-value
  warn("p-value is greater than the indicated p-value", InterpolationWarning)


La série temporelle des actifs aux Etats-Unis est stationnaire intégrée à l'ordre 1 selon les tests KPSS et ADF.

In [38]:
serie1=tocde['USA']['Formation']

In [39]:
kpss_test(serie1)
adf_test(serie1)
serie2 = serie1 - serie1.shift().fillna(0)
kpss_test(serie2)
adf_test(serie2)

KPSS Statistic: 0.8792935265198131
p-value: 0.01
num lags: 13
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
Result: The series is not stationary
ADF Statistic: -0.6611884906481105
p-value: 0.8565481530480973
num lags: 1
Critial Values:
   1% : -3.4870216863700767
   5% : -2.8863625166643136
   10% : -2.580009026141913
Result: The series is not stationary
KPSS Statistic: 0.3092990997033586
p-value: 0.1
num lags: 13
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
Result: The series is stationary
ADF Statistic: -41.07736735499937
p-value: 0.0
num lags: 0
Critial Values:
   1% : -3.4865346059036564
   5% : -2.8861509858476264
   10% : -2.579896092790057
Result: The series is stationary


C:\Users\duran\anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1685: InterpolationWarning: p-value is smaller than the indicated p-value
  warn("p-value is smaller than the indicated p-value", InterpolationWarning)
C:\Users\duran\anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1687: InterpolationWarning: p-value is greater than the indicated p-value
  warn("p-value is greater than the indicated p-value", InterpolationWarning)


La série temporelle de la formation aux Etats-Unis est stationnaire intégrée à l'ordre 1 selon les tests KPSS et AFD.

In [40]:
serie1=tocde['USA']['Conso']
kpss_test(serie1)
adf_test(serie1)
serie2 = serie1 - serie1.shift().fillna(0)
kpss_test(serie2)
adf_test(serie2)

KPSS Statistic: 0.9562502354679252
p-value: 0.01
num lags: 13
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
Result: The series is not stationary
ADF Statistic: -0.5539760704231055
p-value: 0.8810461846151006
num lags: 4
Critial Values:
   1% : -3.4885349695076844
   5% : -2.887019521656941
   10% : -2.5803597920604915
Result: The series is not stationary
KPSS Statistic: 0.32671263492350594
p-value: 0.1
num lags: 13
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
Result: The series is stationary
ADF Statistic: -3.2792113216943397
p-value: 0.015833690928518565
num lags: 3
Critial Values:
   1% : -3.4880216384691867
   5% : -2.8867966864160075
   10% : -2.5802408234244947
Result: The series is stationary


C:\Users\duran\anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1685: InterpolationWarning: p-value is smaller than the indicated p-value
  warn("p-value is smaller than the indicated p-value", InterpolationWarning)
C:\Users\duran\anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1687: InterpolationWarning: p-value is greater than the indicated p-value
  warn("p-value is greater than the indicated p-value", InterpolationWarning)


In [42]:
serie1=tocde['USA']['Exports']
kpss_test(serie1)
adf_test(serie1)
serie2 = serie1 - serie1.shift().fillna(0)
kpss_test(serie2)
adf_test(serie2)

KPSS Statistic: 0.9535354665350328
p-value: 0.01
num lags: 13
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
Result: The series is not stationary
ADF Statistic: -0.4153590512020413
p-value: 0.9075026091117948
num lags: 1
Critial Values:
   1% : -3.4870216863700767
   5% : -2.8863625166643136
   10% : -2.580009026141913
Result: The series is not stationary
KPSS Statistic: 0.2589481789556083
p-value: 0.1
num lags: 13
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
Result: The series is stationary
ADF Statistic: -21.801809529152372
p-value: 0.0
num lags: 0
Critial Values:
   1% : -3.4865346059036564
   5% : -2.8861509858476264
   10% : -2.579896092790057
Result: The series is stationary


C:\Users\duran\anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1685: InterpolationWarning: p-value is smaller than the indicated p-value
  warn("p-value is smaller than the indicated p-value", InterpolationWarning)
C:\Users\duran\anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1687: InterpolationWarning: p-value is greater than the indicated p-value
  warn("p-value is greater than the indicated p-value", InterpolationWarning)


Les séries des exportations et de la consommation aux Etat-Unis sont stationnaires d'ordre 1. 

In [47]:
serie1=tocde['FRA']['Exports']
kpss_test(serie1)
adf_test(serie1)
serie2 = serie1 - serie1.shift().fillna(0)
kpss_test(serie2)
adf_test(serie2)

KPSS Statistic: 0.9463214923897139
p-value: 0.01
num lags: 13
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
Result: The series is not stationary
ADF Statistic: -0.59649256401672
p-value: 0.8717725957184028
num lags: 4
Critial Values:
   1% : -3.4885349695076844
   5% : -2.887019521656941
   10% : -2.5803597920604915
Result: The series is not stationary
KPSS Statistic: 0.3247951370975108
p-value: 0.1
num lags: 13
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
Result: The series is stationary
ADF Statistic: -5.326476050950481
p-value: 4.824924943028646e-06
num lags: 3
Critial Values:
   1% : -3.4880216384691867
   5% : -2.8867966864160075
   10% : -2.5802408234244947
Result: The series is stationary


C:\Users\duran\anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1685: InterpolationWarning: p-value is smaller than the indicated p-value
  warn("p-value is smaller than the indicated p-value", InterpolationWarning)
C:\Users\duran\anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1687: InterpolationWarning: p-value is greater than the indicated p-value
  warn("p-value is greater than the indicated p-value", InterpolationWarning)


In [48]:
serie1=tocde['FRA']['PIB']
kpss_test(serie1)
adf_test(serie1)
serie2 = serie1 - serie1.shift().fillna(0)
kpss_test(serie2)
adf_test(serie2)

KPSS Statistic: 0.946316714699186
p-value: 0.01
num lags: 13
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
Result: The series is not stationary
ADF Statistic: -0.7398047709641851
p-value: 0.8361681058301742
num lags: 3
Critial Values:
   1% : -3.4880216384691867
   5% : -2.8867966864160075
   10% : -2.5802408234244947
Result: The series is not stationary
KPSS Statistic: 0.3431817814545472
p-value: 0.1
num lags: 13
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
Result: The series is stationary
ADF Statistic: -4.235625284564322
p-value: 0.0005731467640234134
num lags: 2
Critial Values:
   1% : -3.487517288664615
   5% : -2.8865777180380032
   10% : -2.5801239192052012
Result: The series is stationary


C:\Users\duran\anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1685: InterpolationWarning: p-value is smaller than the indicated p-value
  warn("p-value is smaller than the indicated p-value", InterpolationWarning)
C:\Users\duran\anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1687: InterpolationWarning: p-value is greater than the indicated p-value
  warn("p-value is greater than the indicated p-value", InterpolationWarning)


In [50]:
serie1=tocde['FRA']['Chomage']
kpss_test(serie1)
adf_test(serie1)
serie2 = serie1 - serie1.shift().fillna(0)
kpss_test(serie2)
adf_test(serie2)

KPSS Statistic: nan
p-value: nan
num lags: 13
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
Result: The series is stationary


MissingDataError: exog contains inf or nans

Nous avons effectué un certain nombre de tests sur les séries. Parfois, les séries n'était stationnaires qu'à l'ordre 1. Dans ce cas la quelle série devons nous utiliser pour la méthode de contrôle synthétique? La série stationnarisée ou la série originale? 